# Lab 1: Implementing self-editing memory from scratch

In [1]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

In [2]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=openai_api_key
)

## Section 1: Breaking down the LLM context window
### A simple agent's context window

In [3]:
model = "gpt-4o-mini"

In [4]:
system_prompt = "You are a chatbot."

## we use openai's expected format in our messages arugument

In [5]:
# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt}, 
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 
    ]
)
chat_completion.choices[0].message.content

"I'm sorry, but I don't know your name. Can you tell me?"

### Adding memory to the context 


In [6]:
#agent_memory = {"person": "Name: Olivia, Hobbies: Running, Scrapbooking"}
#agent_memory = {"homo sapiens": "Name: Jess, Hobbies: Running, Scrapbooking"}
#agent_memory = {"husband": "Name: Joe, Hobbies: Dancing, Painting"}
agent_memory = {"human": "Name: Sanika, Hobbies: Running, Reading"}

system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

**Note: crazy how I can just say human or person or homo sapiens or husband and it will still derive context from it.**

In [7]:
import json


chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + \
         json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "Suggest multiple activities I'd like. Also, what is my name?"},
    ],
)
chat_completion.choices[0].message.content

"Your name is Sanika! Here are some activities you might enjoy based on your hobbies of running and reading:\n\n1. **Trail Running:** Explore local trails to combine your love of running with the beauty of nature.\n\n2. **Book Club:** Join or start a book club to discuss your favorite reads with others and discover new books.\n\n3. **Running Events:** Participate in local running events or races, such as 5Ks or half-marathons, to challenge yourself and meet fellow runners.\n\n4. **Audiobooks for Runs:** Listen to audiobooks while running to make your workouts more enjoyable and productive.\n\n5. **Writing a Blog:** Start a blog about your running journey or book reviews to share your insights and experiences with others.\n\n6. **Storytelling Runs:** Create a route that tells a story, maybe inspired by a book, and visualize it as you run.\n\n7. **Outdoor Reading:** Find a quiet park or a picturesque spot to read outdoors after a good run.\n\n8. **Running Workshops:** Attend workshops or

## Section 2: Modifing the memory with tools 

### Defining a memory editing tool 


In [8]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory 

In [9]:
agent_memory

{'human': '', 'agent': ''}

In [10]:
core_memory_save("human", "The human's name is Charles")

In [11]:
agent_memory

{'human': "\nThe human's name is Charles", 'agent': ''}

In [12]:
core_memory_save("human", "The human's wife's name was Daiana. His second wife's name is Camilla.")

In [13]:
agent_memory

{'human': "\nThe human's name is Charles\nThe human's wife's name was Daiana. His second wife's name is Camilla.",
 'agent': ''}

In [14]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

**We pass openai's exact tool schema for core_memory_save_metadata**

In [32]:
agent_memory = {"human": ""}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "My name is Sanika."},
        {"role": "user", "content": "Let me briefly introduce myself: I am 28 and work as a Data Engineer. In my free time, I like reading books, running and scrapbooking. I care about climate action and plan to eventually work in the area. My favourite colour is coral, but for clothing, I prefer wearing black, cream and white."}
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xnlbyODunwPj1nTw26q8FT2h', function=Function(arguments='{"section": "human", "memory": "Name: Sanika, Age: 28, Occupation: Data Engineer, Hobbies: Reading books, running, scrapbooking, Interests: Climate action, Future Career Interest: Work in climate action, Favorite color: Coral, Clothing preference: Black, cream, and white."}', name='core_memory_save'), type='function')], annotations=[]))

### Executing the tool 


In [33]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human',
 'memory': 'Name: Sanika, Age: 28, Occupation: Data Engineer, Hobbies: Reading books, running, scrapbooking, Interests: Climate action, Future Career Interest: Work in climate action, Favorite color: Coral, Clothing preference: Black, cream, and white.'}

In [34]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [35]:
agent_memory

{'human': '\nName: Sanika, Age: 28, Occupation: Data Engineer, Hobbies: Reading books, running, scrapbooking, Interests: Climate action, Future Career Interest: Work in climate action, Favorite color: Coral, Clothing preference: Black, cream, and white.'}

### Running the next agent step 


In [36]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "What is my name? What outfit should I wear today?"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

ChatCompletionMessage(content="Your name is Sanika. For today's outfit, considering your clothing preferences of black, cream, and white, you might choose a stylish black top paired with cream or white pants. You could also accessorize with coral accents to reflect your favorite color. How does that sound?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])

## Implementing an agentic loop


In [37]:
agent_memory = {"human": ""}

In [38]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [39]:
def agent_step(user_message): 
    
    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True: 
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)
        
        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            return response.message.content

        # if calling a tool, execute the tool
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "core_memory_save", 
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [40]:
agent_step("My name is Sanika.")

TOOL CALL: Function(arguments='{"section":"human","memory":"The user\'s name is Sanika."}', name='core_memory_save')


'Nice to meet you, Sanika! How can I assist you today?'

In [ ]:
# Try some prompts of your own!